In [18]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import requests

## AQ Stations data

In [2]:
'''
Data downloaded from https://discomap.eea.europa.eu/App/AQViewer/index.html?fqn=Airquality_Dissem.b2g.measurements
'''
stations = pd.read_csv('data/DataExtract.csv')

/tmp/ipykernel_3051999/2645687311.py:4: DtypeWarning: Columns (7,28,30,33,35,37,38,40,51,53,55,57) have mixed types. Specify dtype option on import or set low_memory=False.
  stations = pd.read_csv('data/DataExtract.csv')


In [3]:
stations.columns

Index(['Country', 'B-G Namespace', 'Year', 'Air Quality Network',
       'Air Quality Network Name', 'Timezone', 'Air Quality Station EoI Code',
       'Air Quality Station Nat Code', 'Air Quality Station Name',
       'Sampling Point Id', 'Air Pollutant', 'Longitude', 'Latitude',
       'Altitude', 'Altitude Unit', 'Air Quality Station Area',
       'Air Quality Station Type', 'Operational Activity Begin',
       'Operational Activity End', 'Sample Id', 'Inlet Height',
       'Inlet Height Unit', 'Building Distance', 'Building Distance Unit',
       'Kerb Distance', 'Kerb Distance Unit', 'Distance Source',
       'Distance Source Unit', 'Main Emission Sources', 'Heating Emissions',
       'Heating Emissions Unit', 'Mobile', 'Traffic Emissions',
       'Traffic Emissions Unit', 'Industrial Emissions',
       'Industrial Emissions Unit', 'Municipality', 'Dispersion Local',
       'Dispersion Regional', 'Distance Junction', 'Distance Junction Unit',
       'Heavy Duty Fraction', 'Height 

In [4]:
SELECTED_COLUMNS = ['Air Pollutant', 'Longitude', 'Latitude', 'Altitude', 'Country','Municipality', 'Air Quality Station EoI Code', 'Measurement Method']

In [5]:
stations_all = stations[stations['Year']==2024.0].loc[:,SELECTED_COLUMNS].groupby('Air Quality Station EoI Code')\
    .agg({'Longitude': 'mean', 
          'Latitude': 'mean', 
          'Altitude': 'mean', 
          'Air Pollutant': lambda x: sorted(set(x)),
          'Country': lambda x: list(set(x))[0] if len(set(x))==1 else set(x),
          'Municipality': lambda x: list(set(x))[0] if len(set(x))==1 else set(x),
          'Measurement Method': lambda x: list(set(x))[0] if len(set(x))==1 else set(x)
         })\
    .rename(columns={'Latitude':'lat', 'Longitude':'lon', 
                     'Altitude': 'alt', 'Air Pollutant': 'pollutants', 
                     'Country': 'country', 'Municipality': 'municipality'})

In [6]:
stations_all

,lon,lat,alt,pollutants,country,municipality,Measurement Method
Air Quality Station EoI Code,,,,,,,
AT0003R,11.70028,47.50306,940.0,[O3],Austria,6215 Achenkirch,UV-P
AT0011A,14.28083,48.31556,269.0,[C6H6],Austria,4040 Linz,NaN
AT0032A,14.30139,48.29750,263.0,"[CO, NO2, SO2]",Austria,4020 Linz,"{NDIR, UV-FL, IR-GFC, chemi}"
AT0035A,11.50333,47.27639,560.0,"[CO, NO2, PM10, SO2]",Austria,6060 Hall in Tirol,"{NDIR, BETA, UV-FL, chemi}"
AT0052A,15.10170,47.08010,595.0,"[NO2, O3, SO2]",Austria,8580 Köflach,"{UV-P, UV-FL, chemi}"
...,...,...,...,...,...,...,...
SK0267A,21.25884,48.72627,209.0,"[BaP in PM10, C6H6, CO, NO, NO2, NOX as NO2, P...",Slovakia,Kosice,"{chemi, nan, UV-FL, TEOM, IR-GFC, GC-FID}"
SK0268A,18.84246,48.59956,296.0,"[PM10, PM2.5]",Slovakia,Ziar nad Hronom,TEOM
SK0269A,18.07683,48.30946,143.0,"[BaP in PM10, C6H6, CO, NO, NO2, NOX as NO2, P...",Slovakia,Nitra,"{chemi, NDIR, nan, UV-FL, TEOM, GC-FID, IR-GFC}"


In [7]:
len(list(set(stations_all.municipality.values))), len(list(set(stations_all.country.values)))

(2319, 10)

In [8]:
list(set(stations_all.country.values))

['Romania',
 'Austria',
 'Estonia',
 'Slovakia',
 'Belgium',
 'Sweden',
 'Spain',
 'Poland',
 'France',
 'Germany']

In [33]:
#SELECTED_POLLUTANTS = ['PM10', 'PM2.5', 'PM1', 'CO', 'NO2', 'CO2']
SELECTED_POLLUTANTS = ['PM2.5']

In [34]:
stations_selected = stations[stations['Air Pollutant'].isin(SELECTED_POLLUTANTS)][stations['Year']==2024.0].loc[:,SELECTED_COLUMNS].groupby('Air Quality Station EoI Code')\
    .agg({'Longitude': 'mean', 
          'Latitude': 'mean', 
          'Altitude': 'mean', 
          'Air Pollutant': lambda x: sorted(set(x)),
          'Country': lambda x: list(set(x))[0] if len(set(x))==1 else set(x),
          'Municipality': lambda x: list(set(x))[0] if len(set(x))==1 else set(x),
          'Measurement Method': lambda x: list(set(x))[0] if len(set(x))==1 else set(x)
         })\
    .rename(columns={'Latitude':'lat', 'Longitude':'lon', 
                     'Altitude': 'alt', 'Air Pollutant': 'pollutants', 
                     'Country': 'country', 'Municipality': 'municipality'})

/tmp/ipykernel_3051999/1823285542.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stations_selected = stations[stations['Air Pollutant'].isin(SELECTED_POLLUTANTS)][stations['Year']==2024.0].loc[:,SELECTED_COLUMNS].groupby('Air Quality Station EoI Code')\


In [35]:
stations_selected.shape

(1776, 7)

## Selected stations

In [36]:
stations_europe = stations[stations['Air Pollutant'].isin(SELECTED_POLLUTANTS)][stations['Year']==2024.0].loc[:,SELECTED_COLUMNS].groupby('Air Quality Station EoI Code')\
    .agg({'Longitude': 'mean', 
          'Latitude': 'mean', 
          'Altitude': 'mean', 
          'Air Pollutant': lambda x: sorted(set(x)),
          'Country': lambda x: list(set(x))[0] if len(set(x))==1 else set(x),
          'Municipality': lambda x: list(set(x))[0] if len(set(x))==1 else set(x),
          'Measurement Method': lambda x: list(set(x))[0] if len(set(x))==1 else set(x)
         })\
    .rename(columns={'Latitude':'lat', 'Longitude':'lon', 
                     'Altitude': 'alt', 'Air Pollutant': 'pollutants', 
                     'Country': 'country', 'Municipality': 'municipality'})

/tmp/ipykernel_3051999/1771059903.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stations_europe = stations[stations['Air Pollutant'].isin(SELECTED_POLLUTANTS)][stations['Year']==2024.0].loc[:,SELECTED_COLUMNS].groupby('Air Quality Station EoI Code')\


In [37]:
stations_europe

,lon,lat,alt,pollutants,country,municipality,Measurement Method
Air Quality Station EoI Code,,,,,,,
AT0ENK1,13.67086,48.39164,525.0,[PM2.5],Austria,4761 Enzenkirchen,"{OPC-CMC, nan}"
AT0ILL1,16.76639,47.77028,117.0,[PM2.5],Austria,7142 Illmitz,"{OPC-CMC, nan}"
AT0PIL1,15.93600,48.72120,315.0,[PM2.5],Austria,"2073 Pillersdorf, Gemeinde Zellerndorf",OPC-CMC
AT0ZOE2,14.44091,47.83828,892.0,[PM2.5],Austria,4462 Reichraming,OPC-CMC
AT10001,16.52617,47.84011,165.0,[PM2.5],Austria,7000 Eisenstadt,"{OPC-CMC, nan}"
...,...,...,...,...,...,...,...
SK0267A,21.25884,48.72627,209.0,[PM2.5],Slovakia,Kosice,TEOM
SK0268A,18.84246,48.59956,296.0,[PM2.5],Slovakia,Ziar nad Hronom,TEOM
SK0269A,18.07683,48.30946,143.0,[PM2.5],Slovakia,Nitra,TEOM


In [38]:
# Create GeoDataFrame
stations_points = gpd.GeoDataFrame(
    stations_europe,
    geometry=gpd.points_from_xy(stations_europe.lon, stations_europe.lat),
    crs="EPSG:4326"
)

# Convert from GPS to Conus Albers. 
stations_points = stations_points.to_crs("EPSG:5069")
print (stations_points.shape)
stations_points.head(2)

(1776, 8)


,lon,lat,alt,pollutants,country,municipality,Measurement Method,geometry
Air Quality Station EoI Code,,,,,,,,
AT0ENK1,13.67086,48.39164,525.0,[PM2.5],Austria,4761 Enzenkirchen,"{OPC-CMC, nan}",POINT (6499223.767 7051688.218)
AT0ILL1,16.76639,47.77028,117.0,[PM2.5],Austria,7142 Illmitz,"{OPC-CMC, nan}",POINT (6652986.464 7239207.429)


In [39]:
m = folium.Map()

for tt in stations_points.itertuples():
    lat, lon = tt.lat, tt.lon
    folium.CircleMarker(
        location=[lat, lon],
        radius=1, 
        fill=True,
        fill_opacity=1
        ).add_to(m)

folium.map.LayerControl('topleft', collapsed= False).add_to(m)
m

## Fab Labs data

In [19]:
url = "https://api.fablabs.io/0/labs.json"
r = requests.get(url)
fl = pd.DataFrame(r.json())

In [20]:
COUNTRY_CODES = [ "AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "ES", "FI", 
                 "FR", "GB", "GR", "HR", "HU", "IE", "IS", "IT", "LT", "LU", "LV", 
                 "MT", "NO", "NL", "PL", "PT", "RO", "SE", "SI", "SK"]
len(COUNTRY_CODES)

31

In [21]:
fl.columns

Index(['id', 'name', 'kind_name', 'parent_id', 'blurb', 'description', 'slug',
       'avatar_url', 'header_url', 'address_1', 'address_2', 'city', 'county',
       'postal_code', 'country_code', 'latitude', 'longitude', 'address_notes',
       'phone', 'email', 'capabilities', 'activity_status', 'links'],
      dtype='object')

In [22]:
fablab_locations = fl[fl['country_code'].isin(COUNTRY_CODES)][fl['activity_status'] == 'active'].loc[:, ['latitude', 'longitude']]\
    .rename(columns={'latitude':'lat', 'longitude':'lon'})
fablab_locations.dropna(inplace=True)
fablab_locations.head(2)

/tmp/ipykernel_3051999/4220986282.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fablab_locations = fl[fl['country_code'].isin(COUNTRY_CODES)][fl['activity_status'] == 'active'].loc[:, ['latitude', 'longitude']]\


,lat,lon
13,65.661713,-20.291235
20,50.733312,1.600563


In [23]:
# Create GeoDataFrame.
fl_points = gpd.GeoDataFrame(
    fablab_locations,
    geometry=gpd.points_from_xy(fablab_locations.lon, fablab_locations.lat),
    crs="EPSG:4326"
)

# Convert from GPS to Conus Albers. 
fl_points = fl_points.to_crs("EPSG:5069")
print (fl_points.shape)
fl_points.head(2)

(620, 3)


,lat,lon,geometry
13,65.661713,-20.291235,POINT (3810969.238 6202720.942)
20,50.733312,1.600563,POINT (5862878.035 6384315.156)


In [24]:
m = folium.Map()

for tt in fl_points.itertuples():
    lat, lon = tt.lat, tt.lon
    folium.CircleMarker(
        location=[lat, lon],
        radius=1, 
        fill=True,
        fill_opacity=1
        ).add_to(m)

folium.map.LayerControl('topleft', collapsed= False).add_to(m)
m

## Joint data

In [27]:
fl_points.shape, stations_europe.shape

((620, 3), (3903, 7))

In [41]:
m = folium.Map()

for tt in fl_points.itertuples():
    lat, lon = tt.lat, tt.lon
    folium.CircleMarker(
        location=[lat, lon],
        radius=5, 
        color = '#880000',
        fill=True,
        fill_opacity=1
        ).add_to(m)
    
for tt in stations_points.itertuples():
    lat, lon = tt.lat, tt.lon
    folium.CircleMarker(
        location=[lat, lon],
        radius=3, 
        fill=True,
        fill_opacity=1
        ).add_to(m)
    
#folium.map.LayerControl('topleft', collapsed= False).add_to(m)
m

In [40]:
# Perform spatial join. Covert meters to miles. 
gdf = gpd.sjoin_nearest(stations_points, fl_points, 
                        how="left", distance_col="meters")
gdf["miles"] = gdf["meters"] * 0.000621371
gdf["km"] = gdf["meters"] * 0.001

# Get min, max and average distance to FabLabs.
min_dist = gdf.km.min()
max_dist = gdf.km.max()
avg_dist = gdf.km.mean()

print(f"min. distance to FabLab: {min_dist}")
print(f"max. distance to FabLab: {max_dist}")
print(f"avg. distance to FabLab: {avg_dist}")

min. distance to FabLab: 0.05049540032487657
max. distance to FabLab: 6117.3017954395245
avg. distance to FabLab: 97.68552180885155


In [42]:
gdf

,lon_left,lat_left,alt,pollutants,country,municipality,Measurement Method,geometry,index_right,lat_right,lon_right,meters,miles,km
Air Quality Station EoI Code,,,,,,,,,,,,,,
AT0ENK1,13.67086,48.39164,525.0,[PM2.5],Austria,4761 Enzenkirchen,"{OPC-CMC, nan}",POINT (6499223.767 7051688.218),1026,48.042766,14.416456,67854.327523,42.162711,67.854328
AT0ILL1,16.76639,47.77028,117.0,[PM2.5],Austria,7142 Illmitz,"{OPC-CMC, nan}",POINT (6652986.464 7239207.429),1180,47.838168,16.250671,39637.612423,24.629663,39.637612
AT0PIL1,15.93600,48.72120,315.0,[PM2.5],Austria,"2073 Pillersdorf, Gemeinde Zellerndorf",OPC-CMC,POINT (6532450.3 7221289.984),1237,48.228082,16.400166,64407.659337,40.021052,64.407659
AT0ZOE2,14.44091,47.83828,892.0,[PM2.5],Austria,4462 Reichraming,OPC-CMC,POINT (6578296.995 7080207.783),1026,48.042766,14.416456,22629.837629,14.061525,22.629838
AT10001,16.52617,47.84011,165.0,[PM2.5],Austria,7000 Eisenstadt,"{OPC-CMC, nan}",POINT (6639029.271 7225304.964),1180,47.838168,16.250671,20782.847418,12.913859,20.782847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SK0267A,21.25884,48.72627,209.0,[PM2.5],Slovakia,Kosice,TEOM,POINT (6673256.487 7591421.737),800,50.030774,21.999526,153090.751622,95.126153,153.090752
SK0268A,18.84246,48.59956,296.0,[PM2.5],Slovakia,Ziar nad Hronom,TEOM,POINT (6624702.639 7417572.824),1736,47.507753,19.062310,121502.316780,75.498016,121.502317
SK0269A,18.07683,48.30946,143.0,[PM2.5],Slovakia,Nitra,TEOM,POINT (6634024.571 7352722.054),2579,48.150080,17.114219,74281.743002,46.156521,74.281743


In [43]:
# Create dfgroups DataFrame. 
dfgroups = pd.DataFrame({
    "proximity": ["very close", "close", "medium", "far"],
    "km": [0., 5., 25., 50.]
})

# Assign group to each location.
gdf_prox = gdf.sort_values("km", ascending=True)
gdf_prox = pd.merge_asof(gdf_prox, dfgroups, on="km", direction="backward")

gdf.head(10)

,lon_left,lat_left,alt,pollutants,country,municipality,Measurement Method,geometry,index_right,lat_right,lon_right,meters,miles,km
Air Quality Station EoI Code,,,,,,,,,,,,,,
AT0ENK1,13.67086,48.39164,525.0,[PM2.5],Austria,4761 Enzenkirchen,"{OPC-CMC, nan}",POINT (6499223.767 7051688.218),1026,48.042766,14.416456,67854.327523,42.162711,67.854328
AT0ILL1,16.76639,47.77028,117.0,[PM2.5],Austria,7142 Illmitz,"{OPC-CMC, nan}",POINT (6652986.464 7239207.429),1180,47.838168,16.250671,39637.612423,24.629663,39.637612
AT0PIL1,15.93600,48.72120,315.0,[PM2.5],Austria,"2073 Pillersdorf, Gemeinde Zellerndorf",OPC-CMC,POINT (6532450.3 7221289.984),1237,48.228082,16.400166,64407.659337,40.021052,64.407659
AT0ZOE2,14.44091,47.83828,892.0,[PM2.5],Austria,4462 Reichraming,OPC-CMC,POINT (6578296.995 7080207.783),1026,48.042766,14.416456,22629.837629,14.061525,22.629838
AT10001,16.52617,47.84011,165.0,[PM2.5],Austria,7000 Eisenstadt,"{OPC-CMC, nan}",POINT (6639029.271 7225304.964),1180,47.838168,16.250671,20782.847418,12.913859,20.782847
AT10003,17.07032,48.10915,136.0,[PM2.5],Austria,2421 Kittsee,NaN,POINT (6626857.218 7274397.267),2369,48.144568,17.111663,4987.998941,3.099398,4.987999
AT2KA21,14.31806,46.62556,445.0,[PM2.5],Austria,9020 Klagenfurt,"{nephelometry+beta, nan}",POINT (6697591.744 7018260.923),2533,46.614088,14.266581,4155.435657,2.582067,4.155436
AT2KA71,14.29914,46.62662,445.0,[PM2.5],Austria,9020 Klagenfurt,"{nephelometry+beta, nan}",POINT (6696904.432 7016974.37),2533,46.614088,14.266581,2861.627927,1.778133,2.861628
AT2WO15,14.84346,46.83498,460.0,[PM2.5],Austria,9400 Wolfsberg,nephelometry+beta,POINT (6692257.919 7064460.984),2533,46.614088,14.266581,50575.025384,31.425854,50.575025


In [44]:
gdf_prox.proximity.value_counts().sort_index()

proximity
close         388
far           715
medium        351
very close    355
Name: count, dtype: int64